In [2]:
import numpy as np
import pandas as pd
import hvplot.pandas
from cryptocmd import CmcScraper

In [3]:
scraper=CmcScraper('BTC')
scrapereth=CmcScraper('ETH')
scraperada=CmcScraper('ADA')
scraperlink=CmcScraper('LINK')
scraperxrp=CmcScraper('XRP')

In [4]:
btcdata=scraper.get_dataframe()
ethdata=scrapereth.get_dataframe()
adadata=scraperada.get_dataframe()
linkdata=scraperlink.get_dataframe()
xrpdata=scraperxrp.get_dataframe()

In [5]:
btcdata.drop(columns=['Open','High','Low'],inplace=True)
ethdata.drop(columns=['Open','High','Low'],inplace=True)
adadata.drop(columns=['Open','High','Low'],inplace=True)
linkdata.drop(columns=['Open','High','Low'],inplace=True)
xrpdata.drop(columns=['Open','High','Low'],inplace=True)

In [6]:
btcdata.set_index('Date',inplace=True)
ethdata.set_index('Date',inplace=True)
adadata.set_index('Date',inplace=True)
linkdata.set_index('Date',inplace=True)
xrpdata.set_index('Date',inplace=True)

In [9]:
joineddata=pd.concat([btcdata,ethdata,adadata,linkdata,xrpdata], axis=1, keys=['BTC', 'ETH','ADA','LINK','XRP'])

In [7]:
closed_data=pd.DataFrame()

In [10]:
closed_data['BTC']=joineddata['BTC']['Close']
closed_data['BTC']=joineddata['BTC']['Close']
closed_data['ETH']=joineddata['ETH']['Close']
closed_data['ADA']=joineddata['ADA']['Close']
closed_data['LINK']=joineddata['LINK']['Close']
closed_data['XRP']=joineddata['XRP']['Close']

In [13]:
closed_data.dropna(inplace=True)

In [16]:
#pct change
closed_pct_change=closed_data.pct_change().dropna()

In [22]:
# mean daily return
mean_daily_return=closed_data.pct_change().dropna().mean()

In [24]:
#correlation
correlation=closed_data.pct_change().dropna().corr()

In [26]:
#returns
stock_normed = closed_data/closed_data.iloc[0]

In [27]:
stock_normed.hvplot()

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [28]:
#log
log_ret = np.log(closed_data/closed_data.shift(1))

In [29]:
log_ret.hvplot.hist(bins=100, subplots=True, width=400, group_label='Ticker', grid=True).cols(2)

:NdLayout   [Default]
   :Histogram   [XRP]   (XRP_count)

In [31]:
#log year
log_ret.mean() * 252

BTC     0.409276
ETH     0.416138
ADA     0.791216
LINK    0.808343
XRP     0.292252
dtype: float64

In [32]:
#covariance
log_ret.cov()

,BTC,ETH,ADA,LINK,XRP
BTC,0.001791,0.001697,0.001810,0.001619,0.001470
ETH,0.001697,0.002829,0.002596,0.002506,0.002268
ADA,0.001810,0.002596,0.005625,0.002816,0.003046
LINK,0.001619,0.002506,0.002816,0.005991,0.002412
XRP,0.001470,0.002268,0.003046,0.002412,0.004743


In [33]:
#year covariance
log_ret.cov()*252

,BTC,ETH,ADA,LINK,XRP
BTC,0.451429,0.427745,0.456055,0.407972,0.370444
ETH,0.427745,0.712886,0.654302,0.631537,0.571628
ADA,0.456055,0.654302,1.417568,0.709635,0.767599
LINK,0.407972,0.631537,0.709635,1.509764,0.607767
XRP,0.370444,0.571628,0.767599,0.607767,1.195126


In [36]:
#random weights
np.random.seed(101)

# Stock Columns
print('Token')
print(closed_data.columns)
print('\n')

# Create Random Weights
print('Creating Random Weights')
weights = np.array(np.random.random(5))
print(weights)
print('\n')

# Rebalance Weights
print('Rebalance to sum to 1.0')
weights = weights / np.sum(weights)
print(weights)
print('\n')

# Expected Return
print('Expected Portfolio Return')
exp_ret = np.sum(log_ret.mean() * weights) *252
print(exp_ret)
print('\n')

# Expected Variance
print('Expected Volatility')
exp_vol = np.sqrt(np.dot(weights.T, np.dot(log_ret.cov() * 252, weights)))
print(exp_vol)
print('\n')

# Sharpe Ratio
SR = exp_ret/exp_vol
print('Sharpe Ratio')
print(SR)

Token
Index(['BTC', 'ETH', 'ADA', 'LINK', 'XRP'], dtype='object')


Creating Random Weights
[0.51639863 0.57066759 0.02847423 0.17152166 0.68527698]


Rebalance to sum to 1.0
[0.26182041 0.28933544 0.01443678 0.08696357 0.3474438 ]


Expected Portfolio Return
0.4108206159047122


Expected Volatility
0.7740966250443034


Sharpe Ratio
0.5307097365024682


In [37]:
#multiple runs
num_ports = 15000

all_weights = np.zeros((num_ports,len(closed_data.columns)))
ret_arr = np.zeros(num_ports)
vol_arr = np.zeros(num_ports)
sharpe_arr = np.zeros(num_ports)

for ind in range(num_ports):

    # Create Random Weights
    weights = np.array(np.random.random(5))

    # Rebalance Weights
    weights = weights / np.sum(weights)
    
    # Save Weights
    all_weights[ind,:] = weights

    # Expected Return
    ret_arr[ind] = np.sum((log_ret.mean() * weights) *252)

    # Expected Variance
    vol_arr[ind] = np.sqrt(np.dot(weights.T, np.dot(log_ret.cov() * 252, weights)))

    # Sharpe Ratio
    sharpe_arr[ind] = ret_arr[ind]/vol_arr[ind]

In [38]:
#sharpe return
sharpe_arr.max()

0.781279898470363

In [39]:
#sharpe max
sharpe_arr.argmax()

10846

In [107]:
#weights
all_weights[10846,:]

array([0.25131836, 0.00446901, 0.38230963, 0.35258834, 0.00931465])

In [108]:
#set max ret,max vol
max_sr_ret = ret_arr[10846]
max_sr_vol = vol_arr[10846]

In [109]:
import holoviews as hv

In [110]:
#plot sharpe
scatter = hv.Scatter((vol_arr, ret_arr, sharpe_arr), 'Volatility', ['Return', 'Sharpe Ratio'])
max_sharpe = hv.Scatter([(max_sr_vol,max_sr_ret)])

scatter.opts(color='Sharpe Ratio', cmap='plasma', width=600, height=400, colorbar=True, padding=0.1) *\
max_sharpe.opts(color='red', line_color='black', size=10)

:Overlay
   .Scatter.I  :Scatter   [Volatility]   (Return,Sharpe Ratio)
   .Scatter.II :Scatter   [x]   (y)

In [111]:
#Functionalize Return and SR operations
def get_ret_vol_sr(weights):
    """
    Takes in weights, returns array or return,volatility, sharpe ratio
    """
    weights = np.array(weights)
    ret = np.sum(log_ret.mean() * weights) * 252
    vol = np.sqrt(np.dot(weights.T, np.dot(log_ret.cov() * 252, weights)))
    sr = ret/vol
    return np.array([ret,vol,sr])

In [112]:
from scipy.optimize import minimize

In [113]:
def neg_sharpe(weights):
    return  get_ret_vol_sr(weights)[2] * -1

In [114]:
# Contraints
def check_sum(weights):
    '''
    Returns 0 if sum of weights is 1.0
    '''
    return np.sum(weights) - 1

In [115]:
# By convention of minimize function it should be a function that returns zero for conditions
cons = ({'type':'eq','fun': check_sum})

In [117]:
# 0-1 bounds for each weight
bounds = ((0, 1), (0, 1), (0, 1), (0, 1), (0,1))

In [118]:
# Initial Guess (equal distribution)
init_guess = [0.25,0.25,0.25,0.25,0.25]

In [119]:
# Sequential Least SQuares Programming (SLSQP).
opt_results = minimize(neg_sharpe,init_guess,method='SLSQP',bounds=bounds,constraints=cons)

In [120]:
#results
opt_results

     fun: -0.7864287571841545
     jac: array([-2.03892589e-04,  1.34192914e-01,  7.49737024e-04, -5.19633293e-04,
        2.90699057e-01])
 message: 'Optimization terminated successfully'
    nfev: 37
     nit: 6
    njev: 6
  status: 0
 success: True
       x: array([3.50552389e-01, 1.98680048e-17, 3.21383633e-01, 3.28063978e-01,
       8.46761897e-17])

In [121]:
opt_results.x

array([3.50552389e-01, 1.98680048e-17, 3.21383633e-01, 3.28063978e-01,
       8.46761897e-17])

In [122]:
get_ret_vol_sr(opt_results.x)

array([0.66294496, 0.84298158, 0.78642876])

In [135]:
#efficient frontier
# Our returns go from 0 to somewhere along .6 to .9
# Create a linspace number of points to calculate x on
frontier_y = np.linspace(.4,.8,25) # Change 100 to a lower number for slower computers!

In [136]:
def minimize_volatility(weights):
    return  get_ret_vol_sr(weights)[1] 

In [137]:
frontier_volatility = []

for possible_return in frontier_y:
    # function for return
    cons = ({'type':'eq','fun': check_sum},
            {'type':'eq','fun': lambda w: get_ret_vol_sr(w)[0] - possible_return})
    
    result = minimize(minimize_volatility,init_guess,method='SLSQP',bounds=bounds,constraints=cons)
    
    frontier_volatility.append(result['fun'])

In [138]:
scatter * hv.Curve((frontier_volatility, frontier_y)).opts(color='green', line_dash='dashed')

:Overlay
   .Scatter.I :Scatter   [Volatility]   (Return,Sharpe Ratio)
   .Curve.I   :Curve   [x]   (y)

### documentation

#### https://examples.pyviz.org/portfolio_optimizer/portfolio.html
